In [364]:
import requests

#compiling all content on the healthy recipe pages on NYT Cooking
healthyDinners = requests.get('https://cooking.nytimes.com/68861692-nyt-cooking/2110373-healthy-weeknight-dinners')
healthyLunches = requests.get('https://cooking.nytimes.com/68861692-nyt-cooking/2110355-healthy-weekday-lunches')
healthyBreakfast = requests.get('https://cooking.nytimes.com/68861692-nyt-cooking/16596-healthy-breakfast-ideas')


In [365]:
import re

#extracting the recipe links from all the pages for healthy

healthyRecipeLinks = []

pattern = re.compile(r'https://cooking\.nytimes\.com/recipes/([^"]*)') #regex string to extract recipe links from the recipe webpages

healthyLunchMatches = pattern.findall(str(healthyLunches.content)) 
healthyDinnerMatches = pattern.findall(str(healthyDinners.content)) 
healthyBreakfastMatches = pattern.findall(str(healthyBreakfast.content))

#list of every link for healthy (not filtered for 5 stars yet)
allMatches = healthyLunchMatches + healthyDinnerMatches + healthyBreakfastMatches

#after this for loop healthyRecipeLinks is now a list of the links of all 5 star healthy recipes
for match in allMatches:
    link = "https://cooking.nytimes.com/recipes/" + match
    content = requests.get(link)
    if(get_star_review(content) == "5"):
        healthyRecipeLinks.append("https://cooking.nytimes.com/recipes/" + match)

#turning into a set gets rid of the duplicate links
healthyRecipesSet = set(healthyRecipeLinks)

In [366]:
print(len(healthyRecipesSet))

42


In [367]:
#web crawler for healthy 

firstHealthy = '/1021960-sheet-pan-roasted-mushrooms-and-spinach'
healthyLinksTried = []
healthyLinks = list(healthyRecipesSet)

healthyCounter = 1


patternHealthy = re.compile('"keywords":"(?:[^"]*healthy[^"]*),')
websitePattern = re.compile(r'href="/recipes/(.*?)"')

print("working")

def doLinkStuffHealthy(link, counter):
    #print(link)
    
    if link in healthyLinksTried:
        fullLink = link
    else:
        fullLink = "https://cooking.nytimes.com/recipes/" + link
    #print(fullLink)
    if fullLink not in healthyLinksTried:
        healthyLinksTried.append(fullLink)
    data = requests.get(fullLink)
    
    
    if bool(patternHealthy.search(str(data.content))):
        if get_star_review(data) == "5" or get_star_review(data) == "4":
            match = patternHealthy.search(str(data.content))
            if fullLink not in healthyLinks:
                healthyLinks.append(fullLink)
    
    websiteMatches = websitePattern.findall(str(data.content))
        
    for match in websiteMatches:
        fullMatch = "https://cooking.nytimes.com/recipes/" + match
        if fullMatch not in healthyLinksTried:
            counter = 1
            return match + "," + str(counter)
    if fullLink == healthyLinksTried[0]:
        print("all links tried sorry")
        return "all links tried sorry,1"
    else:
        counter = counter + 1
        stringCounter = str(counter)
        newIndex = -1 - counter
        return str(healthyLinksTried[newIndex]) + "," + stringCounter
    

currentLink = doLinkStuffHealthy(firstHealthy, healthyCounter)
currentCounter = 1
while len(healthyLinks) < 50 and currentLink != "all links tried sorry":
   
    newLinkandCounter = doLinkStuffHealthy(currentLink, currentCounter)
    splitUp = newLinkandCounter.split(",")
    newLink = splitUp[0]
    newCounter = splitUp[1]
    currentLink = newLink
    currentCounter = int(newCounter)

    
print("all done")        




working
all done


In [368]:
healthySet = set(healthyLinks)
print(len(healthySet))

50


In [369]:
allHealthyReviews = []

for link in healthySet:
    content = requests.get(link)
    allHealthyReviews.extend(get_review_text(content))

In [370]:
#allHealthyReviews is final result ie. all reviews of all 5 star healthy recipes
print("Total Healthy Recipes Used: " + str(len(healthySet)))
print("Total Healthy Reviews: " + str(len(allHealthyReviews)))

Total Healthy Recipes Used: 50
Total Healthy Reviews: 2788


In [371]:
#THE FOLLOWING IS FOR DESSERT RECIPES

In [372]:
#web crawler for dessert 

firstDessert = '/1015819-chocolate-chip-cookies'
dessertLinksTried = []
dessertLinks = []

myCounter = 1


patternDessert = re.compile('recipeCategory":"dessert"')
websitePattern = re.compile(r'href="/recipes/(.*?)"')

print("working")

def doLinkStuffDessert(link, counter):
    #print(link)
    
    if link in dessertLinksTried:
        fullLink = link
    else:
        fullLink = "https://cooking.nytimes.com/recipes/" + link
    #print(fullLink)
    if fullLink not in dessertLinksTried:
        dessertLinksTried.append(fullLink)
    data = requests.get(fullLink)
    
    
    if bool(patternDessert.search(str(data.content))):
        if get_star_review(data) == "5" or get_star_review(data) == "4":
            match = patternDessert.search(str(data.content))
            #print(match.group())
            dessertLinks.append(fullLink)
    
    websiteMatches = websitePattern.findall(str(data.content))
   # print(websiteMatches)
        
    for match in websiteMatches:
        fullMatch = "https://cooking.nytimes.com/recipes/" + match
        if fullMatch not in dessertLinksTried:
            counter = 1
            return match + "," + str(counter)
    if fullLink == dessertLinksTried[0]:
        print("all links tried sorry")
        return "all links tried sorry,1"
    else:
        counter = counter + 1
        stringCounter = str(counter)
        newIndex = -1 - counter
       # print("counter " + str(counter))
        #print("newIndex" + str(newIndex))
        #print(str(dessertLinksTried[newIndex])) 
        #print(dessertLinksTried[0])
        return str(dessertLinksTried[newIndex]) + "," + stringCounter
    

currentLink = doLinkStuffDessert(firstDessert, myCounter)
currentCounter = 1
while len(dessertLinks) < 50 and currentLink != "all links tried sorry":
    #print(currentLink)
    
    
    newLinkandCounter = doLinkStuffDessert(currentLink, currentCounter)
    splitUp = newLinkandCounter.split(",")
    newLink = splitUp[0]
    newCounter = splitUp[1]
    currentLink = newLink
    currentCounter = int(newCounter)

    
print("all done")

working
all done


In [373]:
dessertLinksSet = set(dessertLinks)

In [374]:
allDessertReviews = []

for link in dessertLinksSet:
    content = requests.get(link)
    allDessertReviews.extend(get_review_text(content))

In [375]:
#allDessertReviews is final result ie. all reviews of all 5 star healthy recipes
print("Total Dessert Recipes Used: " + str(len(dessertLinksSet)))
print("Total Dessert Reviews: " + str(len(allDessertReviews)))

Total Dessert Recipes Used: 51
Total Dessert Reviews: 2704


In [376]:
#HELPER FUNCTIONS BELOW

In [377]:
def get_star_review(websiteData):
    
    pattern = re.compile(r'"ratingValue":(\d+)')
    match = pattern.search(str(websiteData.content))   
    
    return match[1]

print(get_star_review(data))

4


In [378]:
def get_review_text(websiteData): #returns a string of all reviews for a dish
    
    all_reviews = []

    pattern = re.compile(r'"reviewBody":\s*"([^"]*)"') #regex string to extract user reviews of dish
    matches = pattern.findall(str(websiteData.content)) 
    
    for match in matches:
        all_reviews.append(match)
    
    return all_reviews

#print(get_review_text(data))